# Recommendation System

## User-Based Collaborative Filtering

## Notebook by [WenyiXu](https://github.com/xuwenyihust)

### Loading the data

In [12]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats.stats import pearsonr

from sklearn import cross_validation

fulldf=pd.read_csv("https://raw.githubusercontent.com/xuwenyihust/Recommendation-Systems/master/data/amazon-fine-foods.csv")
print(fulldf.head(5))

    Id   ProductId          UserId                      ProfileName  \
0  1.0  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1  2.0  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2  3.0  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3  4.0  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4  5.0  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score          Time  \
0                   1.0                     1.0    5.0  1.303862e+09   
1                   0.0                     0.0    1.0  1.346976e+09   
2                   1.0                     1.0    4.0  1.219018e+09   
3                   3.0                     3.0    2.0  1.307923e+09   
4                   0.0                     0.0    5.0  1.350778e+09   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitalit

C:\Users\Wenyi Xu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (1,2,3,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Train/Test Split

Split the train & test dataset, use the train subset to train the model, and validate the model using test subset.

In [11]:
X = fulldf[['Id', 'ProductId', 'UserId']]
y = fulldf[['Score', 'HelpfulnessNumerator', 'HelpfulnessDenominator']]

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(146604, 3) (146604, 3)
(36652, 3) (36652, 3)


### User-to-user similarity

Define a function to compute the similarities between users.

Based on Pearson correlation.

In [13]:
def com_sim(user0, user1):
    return pearsonr(user0, user1)[0] 

### K nearest users

Given an active user and an item that hasn't been rated by him/her, find the K nearest users to that active one that have rated the item.

In [14]:
def k_nearest(user, item, k):
    all_users = fulldf.UserID.unique()
    
    